# Your Header Title

In [1]:
from typing import Any

In [2]:
import random

In [3]:
import gymnasium as gym

In [4]:
from graph_jsp_env.disjunctive_graph_jsp_env import DisjunctiveGraphJspEnv
from jsp_instance_utils.instances import ft06, ft06_makespan

          

     0100100100100000001111                                            
     0000110011001000000100     ▐███▌         ▟███▛▟███████▛▐███▌      
     1111011011000111100101     ▐███▌        ▟███▛▟███████▛ ▐███▌      
     10▐███▐██▙▟██00▟█▙0100     ▐███▌ ▟███  ▟███▛    ▟███▛  ▐███▌             
     101▐█1▐█1▜▛▐█0▟█▀█▙001     ▐███▌▟████ ▟███▛    ▟███▛   ▐███▌
     00▐███▐█000▐█▟█▛▀▀█▙01     ▐██████▛▐█████▛    ▟████████▐█████████▛ 
     0010010010000000111100     ▐█████▛ ▐████▛    ▟█████████▐████████▛ 


     ▐█▀▜▙█▙   ███████▐█ ▐█    ▟█▙   ▟█▙  ▟███▐█ ▐█▐█▀▀▐█▙ █
     ▐█▄▟▛▜█▙▟▙██ ▐█  ▐████   ▟▛ ▜▙ ▟▛ ▜▙ █▍  ▐████▐█▀▀▐██▙█
     ▐█ ▜▙ ▜█▛▜██ ▐█  ▐█ ▐█  ▟█▛▀▜█▙█▛▀▜█▙▜███▐█ ▐█▐█▆▆▐█ ▜█
           ▐█  ▐█▐█▙ █▐███▜█▙ ▟███▀▀▐█▀▜▙▟█▀▜█▐███▐█████▙ ▟▛
           ▐█  ▐█▐██▙█ ▐█  ▜█▄█▛▐█▀▀▐█▄▟▛▜█▆▆▄ ▐█  ▐█  ▜█▄▛ 
            ▜███▛▐█ ▜█▐███  ▜█▛ ▐███▐█ ▜▙▐█▆▆▛▐███ ▐█   ██  

    
    Disjunctive Graph Job Shop Problem Environment
         

    Version:    0.3.3


In [5]:
from gymcts.gymcts_agent import GymctsAgent
from gymcts.gymcts_env_abc import GymctsABC

In [6]:
from gymcts.logger import log

In [7]:
class GraphJspGYMCTSWrapper(GymctsABC, gym.Wrapper):

    def __init__(self, env: DisjunctiveGraphJspEnv):
        gym.Wrapper.__init__(self, env)

    def load_state(self, state: Any) -> None:
        self.env.reset()
        for action in state:
            self.env.step(action)

    def is_terminal(self) -> bool:
        return self.env.unwrapped.is_terminal()

    def get_valid_actions(self) -> list[int]:
        return list(self.env.unwrapped.valid_actions())

    def rollout(self) -> float:
        terminal = env.is_terminal()

        if terminal:
            lower_bound = env.unwrapped.reward_function_parameters['scaling_divisor']
            return - env.unwrapped.get_makespan() / lower_bound + 2

        reward = 0
        while not terminal:
            action = random.choice(self.get_valid_actions())
            obs, reward, terminal, truncated, _ = env.step(action)

        return reward + 2

    def get_state(self) -> Any:
        return env.unwrapped.get_action_history()

In [8]:
if __name__ == '__main__':
    log.setLevel(20)

    env_kwargs = {
        "jps_instance": ft06,
        "default_visualisations": ["gantt_console", "graph_console"],
        "reward_function_parameters": {
            "scaling_divisor": ft06_makespan
        },
        "reward_function": "nasuta",
    }

    env = DisjunctiveGraphJspEnv(**env_kwargs)
    env.reset()

    env = GraphJspGYMCTSWrapper(env)

    agent = GymctsAgent(
        env=env,
        clear_mcts_tree_after_step=True,
        render_tree_after_step=True,
        exclude_unvisited_nodes_from_render=True,
        number_of_simulations_per_step=50,
    )

    root = agent.search_root_node.get_root()

    actions = agent.solve(render_tree_after_step=True)
    for a in actions:
        obs, rew, term, trun, info = env.step(a)

    env.render()
    makespan = env.unwrapped.get_makespan()
    print(f"makespan: {makespan}")

(N=50, Q_v=0.67, best=0.91)
├── (a=0, N=8, Q_v=0.66, best=0.91, ubc=1.16)
│   ├── (a=1, N=2, Q_v=0.82, best=0.91, ubc=1.54)
│   │   └── (a=12, N=1, Q_v=0.73, best=0.73, ubc=1.32)
│   ├── (a=6, N=1, Q_v=0.60, best=0.60, ubc=1.62)
│   ├── (a=12, N=1, Q_v=0.76, best=0.76, ubc=1.78)
│   ├── (a=18, N=1, Q_v=0.56, best=0.56, ubc=1.58)
│   ├── (a=24, N=1, Q_v=0.65, best=0.65, ubc=1.67)
│   └── (a=30, N=1, Q_v=0.62, best=0.62, ubc=1.64)
├── (a=6, N=8, Q_v=0.67, best=0.91, ubc=1.16)
│   ├── (a=0, N=2, Q_v=0.60, best=0.91, ubc=1.32)
│   │   └── (a=12, N=1, Q_v=0.29, best=0.29, ubc=0.88)
│   ├── (a=7, N=1, Q_v=0.78, best=0.78, ubc=1.80)
│   ├── (a=12, N=1, Q_v=0.71, best=0.71, ubc=1.73)
│   ├── (a=18, N=1, Q_v=0.76, best=0.76, ubc=1.78)
│   ├── (a=24, N=1, Q_v=0.60, best=0.60, ubc=1.62)
│   └── (a=30, N=1, Q_v=0.85, best=0.85, ubc=1.87)
├── (a=12, N=7, Q_v=0.62, best=0.85, ubc=1.15)
│   ├── (a=0, N=1, Q_v=0.62, best=0.62, ubc=1.60)
│   ├── (a=6, N=1, Q_v=0.85, best=0.85, ubc=1.84)
│   ├── (a=13, 

[16:52:32] INFO     selected action 18 after 50 simulations.

           INFO     current action list: [18]

(N=50, Q_v=0.66, best=0.91)
├── (a=0, N=8, Q_v=0.65, best=0.76, ubc=1.15)
│   ├── (a=1, N=1, Q_v=0.71, best=0.71, ubc=1.73)
│   ├── (a=6, N=1, Q_v=0.55, best=0.55, ubc=1.56)
│   ├── (a=12, N=2, Q_v=0.68, best=0.75, ubc=1.40)
│   │   └── (a=30, N=1, Q_v=0.62, best=0.62, ubc=1.21)
│   ├── (a=19, N=1, Q_v=0.62, best=0.62, ubc=1.64)
│   ├── (a=24, N=1, Q_v=0.58, best=0.58, ubc=1.60)
│   └── (a=30, N=1, Q_v=0.65, best=0.65, ubc=1.67)
├── (a=6, N=9, Q_v=0.69, best=0.91, ubc=1.16)
│   ├── (a=0, N=1, Q_v=0.49, best=0.49, ubc=1.54)
│   ├── (a=7, N=1, Q_v=0.45, best=0.45, ubc=1.50)
│   ├── (a=12, N=2, Q_v=0.76, best=0.91, ubc=1.50)
│   │   └── (a=19, N=1, Q_v=0.62, best=0.62, ubc=1.21)
│   ├── (a=19, N=2, Q_v=0.77, best=0.91, ubc=1.51)
│   │   └── (a=30, N=1, Q_v=0.64, best=0.64, ubc=1.22)
│   ├── (a=24, N=1, Q_v=0.76, best=0.76, ubc=1.81)
│   └── (a=30, N=1, Q_v=0.76, best=0.76, ubc=1.81)
├── (a=12, N=9, Q_v=0.67, best=0.84, ubc=1.14)
│   ├── (a=0, N=1, Q_v=0.69, best=0.69, ubc=1.74)
│   ├── (a

           INFO     selected action 6 after 50 simulations.

...

(N=50, Q_v=0.95, best=0.95)
└── (a=23, N=49, Q_v=0.95, best=0.95, ubc=1.15)


[16:52:57] INFO     selected action 23 after 50 simulations.

           INFO     current action list: [18, 6, 7, 12, 8, 19, 24, 20, 9, 13, 10, 21, 30, 11, 14, 25, 15, 16, 17,  
                    26, 0, 27, 31, 22, 32, 33, 34, 1, 2, 35, 3, 4, 28, 29, 5, 23]

           INFO     Final action list: [18, 6, 7, 12, 8, 19, 24, 20, 9, 13, 10, 21, 30, 11, 14, 25, 15, 16, 17, 26,
                    0, 27, 31, 22, 32, 33, 34, 1, 2, 35, 3, 4, 28, 29, 5, 23]

Job 0     ●-●-●-●-●-●     Machine 0 █
Job 1     ●-●-●-●-●-●     Machine 1 █
Job 2     ●-●-●-●-●-●     Machine 2 █
Job 3     ●-●-●-●-●-●     Machine 3 █
Job 4     ●-●-●-●-●-●     Machine 4 █
Job 5     ●-●-●-●-●-●     Machine 5 █
         ╔═══════════════════════════════════════════════════════╗
Job 0    ║     █    ███  █████████████              ██     ██████║ Machine 0   █
Job 1    ║     ████████████████████████████████████████████      ║ Machine 1   █
Job 2    ║██████████████████████████ ██████                      ║ Machine 2   █
Job 3    ║██████████                ███████     ████████████████ ║ Machine 3   █
Job 4    ║                 ███████████     █████████   ███ █     ║ Machine 4   █
Job 5    ║             ███████████████████████         █████     ║ Machine 5   █
         ╚╦════╤════╤════╤════╤════╦════╤════╤════╤════╤════╦════╝
          0.0                      26.8                     53.6 
makespan: 58
